In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

url = 'https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20countries%20and%20dependencies%20and%20their%20capitals%20in%20native%20languages%20-%20Wikipedia.html'

povezava = requests.get(url)
spletna_stran = BeautifulSoup(povezava.content, 'html.parser') #prenos HTML vsebine s spletne strani

tabele = spletna_stran.find_all('table', {'class': 'wikitable'}) #iskanje vseh tabel na strani z razredom 'wikitable'

seznami_podatkov = [pd.read_html(str(tabela))[0] for tabela in tabele] # pretvorba vsake tabele v Pandas DataFramež

zdruzena_tabela = pd.concat(seznami_podatkov, ignore_index=True)
print(zdruzena_tabela)


nova_tabela = zdruzena_tabela[['Country (exonym)', 'Capital (exonym)']].copy()
nova_tabela.columns = ['drzava', 'ime']

# Funkcija za čiščenje držav
def ocisti_drzavo(drzava):
    if isinstance(drzava, str):  # Preverimo, če je vrednost niz (string)
        drzava = re.sub(r'\[.*?\]', '', drzava)  # Odstrani oglati oklepaji
        drzava = re.sub(r'\(.*?\)', '', drzava)  # Odstrani okrogli oklepaji
        return drzava.strip()  # Odstrani odvečne presledke
    return drzava  # Če je NaN, pustimo nespremenjeno

# Funkcija za čiščenje glavnih mest
def ocisti_mesto(mesto):
    if isinstance(mesto, str):  # Preverimo, če je niz (string)
        mesto = re.split(r'\(|\[', mesto)[0]  # Odstrani vse za prvim oklepajem (okrogel ali oglati)
        return mesto.strip()  # Odstrani odvečne presledke
    return mesto
    
# Uporaba funkcij za čiščenje
nova_tabela['drzava'] = nova_tabela['drzava'].apply(ocisti_drzavo)
nova_tabela['ime'] = nova_tabela['ime'].apply(ocisti_mesto)
print(nova_tabela)

# v csv 
glavna_mesta = "drzave_in_glavna_mesta.csv"
nova_tabela.to_csv(glavna_mesta, index=False)
print(f"Podatki so shranjeni v datoteko: {glavna_mesta}")